In [8]:
import os
from os import listdir
from os.path import isfile, join
import sys
import pandas as pd
import random
import numpy as np
import pickle as pkl
from operator import add 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression, Lasso
from collections import Counter

from utils import *
import CFS

import TCA

import copy

In [9]:
def prepare_data(project, metric):
    data_path = '../data/700/merged_data_original/' + project + '.csv'
    data_df = pd.read_csv(data_path)
    data_df.rename(columns = {'Unnamed: 0':'id'},inplace = True)

    for col in ['id', 'commit_hash', 'release']:
        if col in data_df.columns:
            data_df = data_df.drop([col], axis = 1)
    data_df = data_df.dropna()
    y = data_df.Bugs
    X = data_df.drop(['Bugs'],axis = 1)
    if metric == 'process':
        X = X[['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
    'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
    'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
    'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr']]
    elif metric == 'product':
        X = X.drop(['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
    'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
    'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
    'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr'],axis = 1)
    else:
        X = X
    df = X
    df['Bugs'] = y
    return df

def apply_cfs(df):
    y = df.Bugs.values
    X = df.drop(labels = ['Bugs'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('Bugs')
    return df[cols],cols

In [88]:
feature_imp = pd.DataFrame()
metric = 'process'
all_imp = {}
for i in range(1):
    _imp = {}
    fold = str(i)
    data_location = 'results/median_data/level_2/fold_' + fold
    bell_df = pd.read_csv(data_location + '/bellwether_cdom_0.csv')
    bellwether = bell_df.bellwether.values.tolist()[0]
    df = prepare_data(bellwether, metric)
    all_cols = df.columns
    new_df, cols = apply_cfs(df)
    y = new_df.Bugs
    X = new_df.drop(labels = ['Bugs'],axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X,y)
    imp = clf.feature_importances_
    for col in all_cols:
        _imp[col] = 0.0
    for j in range(len(cols)-1):
        col = cols[j]
        _imp[col] = imp[j]
    all_imp[i] = _imp

In [89]:
all_imp_df = pd.DataFrame.from_dict(all_imp, orient = 'index')
all_imp_df = round(all_imp_df.sum(), 2)
all_imp_df.to_csv('results/median_data/features/features_01.csv')

In [35]:
attributes = pd.read_pickle('results/attributes/projects_attributes.pkl')
attributes_df = pd.DataFrame.from_dict(attributes, orient = 'index')

In [86]:
feature_imp = pd.DataFrame()
metric = 'process'
all_imp = {}
for i in range(1):
    fold = str(i)
    data_location = 'results/median_data/level_2/fold_' + fold
    bell_df = pd.read_csv(data_location + '/bellwether_cdom_2.csv')
    for cluster in bell_df.id.values.tolist():
        cdom_df = pd.read_csv(data_location + '/' + str(cluster) + '/cdom_latest.csv', index_col = 0)
        projects = cdom_df.index.values.tolist()
        for project in projects:
            try:
                df = prepare_data(project, metric)
                _imp = {}
                all_cols = df.columns
                for col in all_cols:
                    _imp[col] = 0.0
                s_cols = []
                for j  in range(attributes_df.loc[project].shape[0]):
                    col = attributes_df.loc[project].values.tolist()[j]
                    if col == 1:
                        s_cols.append(all_cols[j])
                new_df = df[s_cols]
                y = new_df.Bugs
                X = new_df.drop(labels = ['Bugs'],axis = 1)
                clf = RandomForestClassifier()
                clf.fit(X,y)
                imp = clf.feature_importances_
                for k in range(len(s_cols)-1):
                    col = s_cols[k]
                    _imp[col] = round(imp[k],2)
                all_imp[project] = _imp
            except Exception as e:
                print(e)
                continue

'gradle-retrolambda'
'the-app'
'derive4j'
'jetcache'


In [87]:
all_imp_df = pd.DataFrame.from_dict(all_imp, orient = 'index')
all_imp_df = round(all_imp_df.sum()/all_imp_df.shape[0],2)
all_imp_df.to_csv('results/median_data/features/features_all_1.csv')